# Upgrade 1: Automated Tuning with Optuna
We'll replace the fixed LightGBM settings with an optimization study. Optuna will intelligently test dozens of hyperparameter combinations (learning rate, number of trees, tree depth, regularization, etc.) and find the set that produces the lowest error for your specific data.

Upgrade 2: Advanced Feature Engineering
This is often the most impactful way to boost performance. We need to create features that reveal more signal to the model.

Interaction Features: We'll combine features to capture complex relationships. For example, dam_height relative to surface_area_acres might be more predictive than either feature alone.

Geospatial Features: Since you have latitude and longitude, we can create clusters of dams using a simple algorithm like KMeans. This can capture regional patterns that the state feature might miss.

The Upgraded Code
This complete script integrates Optuna for hyperparameter searching and includes a more advanced feature engineering function. The process for each target variable will now be:

Create advanced features.

Run an Optuna study to find the best model parameters.

Train a final model using those best parameters.

Evaluate and plot the results.

In [4]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error
import matplotlib.pyplot as plt
import openpyxl
import os

# --- Configuration ---
DATA_FILE = '../regression_data.csv'
OPTUNA_TRIALS = 30 # Number of different hyperparameter sets to test

# --- ADVANCED FEATURE ENGINEERING ---
def engineer_features(df):
    """Creates more sophisticated features to improve model performance."""
    df_copy = df.copy()
    
    # Ensure key date columns are numeric, coercing errors
    for col in ['year_completed', 'incident_date_year', 'year_modified']:
        if col in df_copy.columns:
            df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')
            
    # Time-based features
    df_copy['dam_age_at_incident'] = df_copy['incident_date_year'] - df_copy['year_completed']
    year_modified_filled = df_copy['year_modified'].fillna(df_copy['year_completed'])
    df_copy['time_since_modification'] = df_copy['incident_date_year'] - year_modified_filled

    # Interaction & Polynomial Features
    numerical_cols = ['dam_height', 'max_storage_ac_ft', 'surface_area_acres']
    # Filter to only columns that exist in the dataframe
    existing_numerical_cols = [col for col in numerical_cols if col in df_copy.columns]
    if len(existing_numerical_cols) > 1:
        # Use a simple interaction for demonstration
        df_copy['height_x_storage'] = df_copy['dam_height'] * df_copy['max_storage_ac_ft']
        df_copy['storage_per_area'] = df_copy['max_storage_ac_ft'] / (df_copy['surface_area_acres'] + 1e-6)

    # Geospatial Clustering Feature
    if 'latitude' in df_copy.columns and 'longitude' in df_copy.columns:
        coords = df_copy[['latitude', 'longitude']].dropna()
        if not coords.empty:
            kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)
            df_copy['geo_cluster'] = kmeans.fit_predict(coords)
            df_copy['geo_cluster'] = df_copy['geo_cluster'].astype('category')
            
    return df_copy.drop(columns=['year_completed', 'year_modified'], errors='ignore')


INPUT_COLUMNS = [
    'state', 'downstream_hazard_potential', 'owner_type', 'dam_type', 'dam_height',
    'primary_purpose_s', 'eap', 'latitude', 'longitude', 'incident_date_year',
    'max_storage_ac_ft', 'surface_area_acres'
]
NEW_TARGET_COLUMNS = [
    'number_of_people_evacuated', 'number_of_habitable_structures_flooded', 'incident_duration'
]

# --- OPTUNA HYPERPARAMETER TUNING ---
def objective(trial, X, y):
    """The function Optuna will try to minimize."""
    # Define the search space for LightGBM's hyperparameters
    param = {
        'objective': 'regression_l1', # MAE, often more robust to outliers
        'metric': 'rmse',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'verbose': -1,
        'n_jobs': -1,
        'seed': 42,
    }
    
    # Use cross-validation to get a robust error estimate
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    cv_errors = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model = lgb.LGBMRegressor(**param)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[lgb.early_stopping(10, verbose=False)])
        preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        cv_errors.append(rmse)
        
    return np.mean(cv_errors)

# --- MAIN PROCESSING FUNCTION ---
def find_best_model_and_evaluate(X, y, target_name, metrics_list):
    print(f"--- Processing target: {target_name} ---")

    # Prepare data: Convert object columns to category for LightGBM
    for col in X.select_dtypes(include=['object']).columns:
        X[col] = X[col].astype('category')

    # 1. Run Optuna study to find best hyperparameters
    print(f"Running Optuna search for '{target_name}'...")
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, X, y), n_trials=OPTUNA_TRIALS)
    best_params = study.best_params
    print(f"✅ Best parameters found for '{target_name}'.")

    # 2. Train final model on all data with the best parameters
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    final_model = lgb.LGBMRegressor(objective='regression_l1', **best_params, random_state=42, verbose=-1)
    final_model.fit(X_train, y_train)

    # 3. Evaluate the final model
    y_pred = final_model.predict(X_test)
    
    # Calculate all metrics
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Final R2 Score for '{target_name}': {r2:.4f}, MAE: {mae:.4f}")
    
    # Add all metrics to the list for the final report
    metrics_list.append({
        'Model Output': target_name, 'R2_Score': r2, 'MAE': mae,
        'MSE': mean_squared_error(y_test, y_pred),
        'Explained_Variance_Score': explained_variance_score(y_test, y_pred)
    })

    # 4. Plotting and saving results
    plt.figure(figsize=(8, 8))
    plt.scatter(y_test, y_pred, alpha=0.6)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=2)
    plt.title(f'Actual vs. Predicted for {target_name} (Tuned LGBM)')
    plt.xlabel('Actual Values'); plt.ylabel('Predicted Values')
    plt.savefig(f'plot_{target_name}.svg')
    plt.close()

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    df_raw = pd.read_csv(DATA_FILE)
    df_featured = engineer_features(df_raw)
    
    all_inputs = [col for col in df_featured.columns if col not in NEW_TARGET_COLUMNS]
    model_metrics_data = []

    for target in NEW_TARGET_COLUMNS:
        if target not in df_featured.columns or not pd.api.types.is_numeric_dtype(df_featured[target]): continue
        
        current_inputs = [col for col in all_inputs if col in df_featured.columns and col != target]
        temp_df = df_featured[current_inputs + [target]].dropna()
        
        if len(temp_df) < 100:
            print(f"Warning: Skipping '{target}' due to insufficient data ({len(temp_df)} rows).")
            continue

        X_data = temp_df[current_inputs]
        y_data = temp_df[target]
        find_best_model_and_evaluate(X_data, y_data, target, model_metrics_data)

    if model_metrics_data:
        pd.DataFrame(model_metrics_data).to_excel('tuned_lgbm_metrics.xlsx', index=False)
        print("\n✅ All tuned LightGBM metrics saved.")

    print("\nAll tasks complete.")

[I 2025-09-19 10:19:37,260] A new study created in memory with name: no-name-bcfe003e-f40a-4d58-a3c5-e239182325f7


--- Processing target: number_of_people_evacuated ---
Running Optuna search for 'number_of_people_evacuated'...


[I 2025-09-19 10:19:37,650] Trial 0 finished with value: 2.8269649221934645 and parameters: {'n_estimators': 746, 'learning_rate': 0.017723654112605333, 'num_leaves': 182, 'max_depth': 3, 'min_child_samples': 99, 'feature_fraction': 0.8238382607911973, 'bagging_fraction': 0.8254223558810423, 'bagging_freq': 4, 'lambda_l1': 1.3807500916564693e-07, 'lambda_l2': 3.023267780204336}. Best is trial 0 with value: 2.8269649221934645.
[I 2025-09-19 10:19:37,926] Trial 1 finished with value: 2.8487338658251438 and parameters: {'n_estimators': 699, 'learning_rate': 0.004725931666377631, 'num_leaves': 138, 'max_depth': 10, 'min_child_samples': 39, 'feature_fraction': 0.593979357940236, 'bagging_fraction': 0.6355107634343377, 'bagging_freq': 6, 'lambda_l1': 5.653857824094703e-07, 'lambda_l2': 5.069124446471762e-07}. Best is trial 0 with value: 2.8269649221934645.
[I 2025-09-19 10:19:38,104] Trial 2 finished with value: 2.8249439780479495 and parameters: {'n_estimators': 242, 'learning_rate': 0.0868

✅ Best parameters found for 'number_of_people_evacuated'.
Final R2 Score for 'number_of_people_evacuated': 0.0964, MAE: 0.5082


[I 2025-09-19 10:20:02,540] A new study created in memory with name: no-name-ca24cd55-6229-4851-9d1a-7c3de761e483


...

✅ Best parameters found for 'number_of_habitable_structures_flooded'.
Final R2 Score for 'number_of_habitable_structures_flooded': 0.2947, MAE: 0.2374


[I 2025-09-19 10:20:37,806] A new study created in memory with name: no-name-e9352d82-e202-4c28-8727-16d4b85faf60
[I 2025-09-19 10:20:37,949] Trial 0 finished with value: 7.17190790151825 and parameters: {'n_estimators': 822, 'learning_rate': 0.023050614305359485, 'num_leaves': 292, 'max_depth': 8, 'min_child_samples': 72, 'feature_fraction': 0.5401202202563471, 'bagging_fraction': 0.4850514585017404, 'bagging_freq': 1, 'lambda_l1': 2.0996562211050156, 'lambda_l2': 0.3334846326898181}. Best is trial 0 with value: 7.17190790151825.


--- Processing target: incident_duration ---
Running Optuna search for 'incident_duration'...


[I 2025-09-19 10:20:38,106] Trial 1 finished with value: 7.169748848272377 and parameters: {'n_estimators': 237, 'learning_rate': 0.0103169808505798, 'num_leaves': 294, 'max_depth': 6, 'min_child_samples': 49, 'feature_fraction': 0.8458799342902874, 'bagging_fraction': 0.44586361315120016, 'bagging_freq': 2, 'lambda_l1': 3.993568694419276e-06, 'lambda_l2': 0.12236589948877664}. Best is trial 1 with value: 7.169748848272377.
[I 2025-09-19 10:20:38,271] Trial 2 finished with value: 7.1670411167931585 and parameters: {'n_estimators': 959, 'learning_rate': 0.02245064159200716, 'num_leaves': 97, 'max_depth': 3, 'min_child_samples': 40, 'feature_fraction': 0.4872387544808849, 'bagging_fraction': 0.8042501119378667, 'bagging_freq': 4, 'lambda_l1': 6.92079585985291e-05, 'lambda_l2': 3.935863787594371e-07}. Best is trial 2 with value: 7.1670411167931585.
[I 2025-09-19 10:20:38,508] Trial 3 finished with value: 7.172427146300849 and parameters: {'n_estimators': 591, 'learning_rate': 0.0073190178

✅ Best parameters found for 'incident_duration'.
Final R2 Score for 'incident_duration': -0.0006, MAE: 1.6262

✅ All tuned LightGBM metrics saved.

All tasks complete.


In [3]:
pip install optuna

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.1 MB 1.2 MB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.1 MB 1.3 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 1.3 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 1.3 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 817.9 kB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 817.9 kB/s eta 0:00:01
   ----------------------------- ---------- 1.6/2.1 MB 806.6 kB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 811.6 kB/s eta 0:00:01
   ------------------------------------